This script creates the supply and demand model at AWC level, time-horizon - daily. 
Script expects these inputs - 
1. Lane assignments - <br>
    i) birth node to nearest AWC <br>
    ii) birth node to nearest PHC <br>
    iii) AWC to nearest PHC
2. Vaccine details - vaccination schedule with start date, vaccination window, Average packed volume (secondary packaging), wastage for each vaccine
3. CCE details - CCE type, capacity and cost

In [1]:
import numpy as np

import pandas as pd

from pathlib import Path

import matplotlib.pyplot as plt 

path_data = Path.cwd().parent / 'data'

from scipy.sparse import csr_matrix, find

In [2]:
birth_awc_df = pd.read_csv(path_data / '02 out_birth_awc_lane_assignment.csv')

birth_phc_df = pd.read_csv(path_data / '02 out_birth_phc_lane_assignment.csv')

awc_phc_df = pd.read_csv(path_data / '02 out_awc_phc_lane_assignment.csv')

vaccine_details_df = pd.read_csv(path_data / '03 in_vaccine_details.csv')

cce_details_df = pd.read_csv(path_data / '03 in_cce_details.csv')

User Input Parameters

In [3]:
awc_supply_frequency = 28    # Frequency of sessions at AWC in days
phc_supply_frequency = 28    # Frequency of replenishment of PHC CCE stock in days

alpha_i = 0.8          # Immunization probability alpha
beta_i = 0.1           # Immunization probability beta
gamma_i = 0.4        # Immunization probability lower limit
delta = 0.5        # Ratio of (distance from nearest AWC)/(distance from nearest PHC), where caregiver's choice probability is equal = 0.5

alpha_d = 4        # Vaccination delay intercept (median vaccination delay for BCG vaccine)   
beta_d = 0.08      # Vaccination delay beta (for every additional day to vaccination start date, median delay increases by beta_d)

## Demand Model

Immunization Probability for each birth node = Vaccination Seeking Probability x Caregiver's Location Choice Probability

Assume that the Caregiver's Location Choice probability is modeled using below equation, where x is the AWC/PHC distance ratio
$$ P(PHC) = \frac{e^x}{1+e^x}
$$

In [4]:
birth_awc_phc_df = pd.merge(birth_awc_df[['Location ID', 'AWC Name', 'AWC_ID', 'Distance', 'Births']].rename(columns = {'Distance':'AWC Distance'}), 
                            birth_phc_df[['Location ID', 'PHC Full Name', 'PHC_ID', 'Distance']].rename(columns = {'Distance': 'PHC Distance'}),
                            on = 'Location ID', how = 'left')

birth_awc_phc_df['AWC PHC Distance Ratio'] = birth_awc_phc_df['AWC Distance'] /  birth_awc_phc_df['PHC Distance']
birth_awc_phc_df['Normalized Distance Ratio'] = (birth_awc_phc_df['AWC PHC Distance Ratio'] - delta)/birth_awc_phc_df['AWC PHC Distance Ratio'].var()

birth_awc_phc_df['P_PHC'] = np.exp(birth_awc_phc_df['Normalized Distance Ratio'])/(1+np.exp(birth_awc_phc_df['Normalized Distance Ratio']))

Vaccination seeking probability is given by the linear relation --
$$ P(VaccinationSeeking) = \alpha - \beta \times d $$
where d is the distance of birth node from nearest AWC or PHC (whichever is closest)

In [5]:
birth_awc_phc_df['Vaccination Seeking Probability'] = alpha_i - birth_awc_phc_df[['AWC Distance', 'PHC Distance']].min(axis = 1) *beta_i
birth_awc_phc_df['Vaccination Seeking Probability'] = birth_awc_phc_df['Vaccination Seeking Probability'].clip(lower = gamma_i)

In [6]:
# birth_awc_phc_df.to_csv(path_data / 'birth_awc_phc_df.csv', index = False)

Number births (births_array) and number immunized at each birth node (immunization_array) <br>
AWC turnout has the distribution - 
$$ Poi(q \times \delta \times \lambda/365) $$

In [7]:
pois = lambda x: np.random.poisson(x/365, 365)

births_array = np.array(list(map(pois, birth_awc_phc_df['Births'].values)))

awc_turnout_array = np.array(list(map(pois, birth_awc_phc_df['Births'] * birth_awc_phc_df['Vaccination Seeking Probability'] * (1 - birth_awc_phc_df['P_PHC']) .values)))
phc_turnout_array = np.array(list(map(pois, birth_awc_phc_df['Births'] * birth_awc_phc_df['Vaccination Seeking Probability'] * birth_awc_phc_df['P_PHC'] .values)))

awc_id_array = birth_awc_phc_df['AWC_ID'].values
phc_id_array = birth_awc_phc_df['PHC_ID'].values

In [8]:
# Histogram of number of births per day
dict(zip(np.histogram(births_array, bins = 4)[1] , np.histogram(births_array, bins = 4)[0]))

{0.0: 1532854, 1.0: 17740, 2.0: 270, 3.0: 21}

In [9]:
print(f"% seeking vaccination : {(awc_turnout_array.sum() + phc_turnout_array.sum()) / births_array.sum():.2%}")
print(f"% not seeking vaccination : {1-(awc_turnout_array.sum() + phc_turnout_array.sum()) / births_array.sum():.2%}\n")
print(f"% seeking vaccination at PHC: {phc_turnout_array.sum()/ (awc_turnout_array.sum() + phc_turnout_array.sum()):.2%}")
print(f"% not seeking vaccination at AWC : {awc_turnout_array.sum()/ (awc_turnout_array.sum() + phc_turnout_array.sum()):.2%}")

% seeking vaccination : 69.47%
% not seeking vaccination : 30.53%

% seeking vaccination at PHC: 16.67%
% not seeking vaccination at AWC : 83.33%


Combine AWC Turnout and PHC turnout into one Session Site level turnout array

In [10]:
birthnode_session_site_demand_array = np.append(awc_turnout_array, phc_turnout_array, axis = 0)
birthnode_session_site_id = np.append(awc_id_array, phc_id_array)

#### Demand-side vaccination delay

Once the caregiver has decided to seek vaccination and selected the location, we now model the delay in vaccination once it is due ie. from Start Week. We assume the delay is a Geometric Random Variable, which implies that on any day after the vaccine is due, the caregiver decides whether to go for vaccination or not as per a Bernoulli trial with probability $p$.

To determine this $p$, we use the data from this paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6996155/#b0060 to get the median delay from vaccine start date for BCG, DTP and Measles. Then we assume that this median delay increases as the vaccine start date increases, eg. BCG start date is 0 days after birth, then median delay is low, but for later vaccines like DTP/Penta, the median delay itself is higher.

From the median delay value, we can estimate the $p$ using the CDF equation for Geometric Distribution   $1 - (1-p)^k = 0.5$   where $k$ is the median delay value

In [11]:
vaccine_details_df['Vaccination Delay Median'] = np.round(alpha_d + beta_d * vaccine_details_df['Start Week'] * 7,0)

vaccine_details_df['Vaccination Delay Geom p'] = 1 - np.power(0.5, 1/vaccine_details_df['Vaccination Delay Median'])

Now we want to get birthnode-vaccine level demand, such that the delay values are applied. We will get non-zero birthnode-day-vaccine combinations and apply appropriate delay factor. 

birthnode_session_site_2year_demand - Duplicate the birthnode_session_site_demand_array (dimension Num_birthnode_session_site x 365) to get a new matrix with dimension Num_birthnode_session_site_turnout x 730. 
Basically we assume that the daily birth numbers in last year were same as current year.
So this helps estimate the demand from babies who are between 0-1 years of age at the start of current year

array_birth_indices - This returns the row-column combinations of birthnode_session_site_2year_demand when 1 or more births happened. Eg BirthNode 1 - Day 5, and so on..

array_birth_count - The number of births that happened corresponding to the index captured in array_birth_indices

In [12]:
birthnode_session_site_2year_demand = np.append(birthnode_session_site_demand_array, birthnode_session_site_demand_array, axis = 1)

array_birth_indices = np.nonzero(birthnode_session_site_2year_demand)
array_birth_count = birthnode_session_site_2year_demand[array_birth_indices]

Now 2 arrays are created - 

1. vaccination_start_array - This is the offset from birth to start date of vaccination.

2. vaccination_delay_array - Based on the Geometric distribution, get the delay in days for every birth-node-day combination. This helps us map for example - For the birth that happened on Birth Node 1 - Day 5, the BCG vaccine is delayed by 4 days, OPV by 7 days and so on.

vaccination_offset_array = vaccination_start_array + vaccination_delay_array

This gives us the total number of days after birth when vaccine is demanded, considering the vaccination start date and corresponding delay

In [13]:
geom = lambda x: np.random.geometric(x, array_birth_indices[0].shape)

vaccination_delay_array = np.array(list(map(geom, vaccine_details_df['Vaccination Delay Geom p'].values))) - 1

vaccination_delay_array = np.transpose(vaccination_delay_array)

vaccination_start_array = np.tile(list(vaccine_details_df['Start Week'] * 7), (len(array_birth_indices[0]),1))

vaccination_offset_array = vaccination_start_array + vaccination_delay_array

Now this vaccination_offset_array needs to be plugged back into the bigger matrix to get the demand (birthnode_vaccine_demand)at vaccine_type x birthnode_sitelevel x 365. This is done by iterating over the array_birth_indices x N_vaccines.

This approach reduces the number of iterations, which would've otherwise been N_session_sites x N_vaccines x 365

In [14]:
birthnode_vaccine_2year_demand = np.zeros([vaccine_details_df.shape[0], birthnode_session_site_2year_demand.shape[0], birthnode_session_site_2year_demand.shape[1]])

for i in range(len(array_birth_indices[0])):
    for vac in range(vaccine_details_df.shape[0]):
        vaccine_day_with_offset = array_birth_indices[1][i] + vaccination_offset_array[i][vac]
        if vaccine_day_with_offset < birthnode_vaccine_2year_demand.shape[2]:
            birth_node_id = array_birth_indices[0][i]
            n_births = array_birth_count[i]
            birthnode_vaccine_2year_demand[vac, birth_node_id, vaccine_day_with_offset] = n_births

birthnode_vaccine_demand = birthnode_vaccine_2year_demand[:,:,365:]

session_site_vaccine_level_demand - Using the above matrix, we now get the demand at session_site-Vaccine level. The dimension of this new matrix is - Num_Vaccines x Num_session_site_turnout x 365)

Aggregate birth-node level data (number of births & number of 'immunized births') at Session Sitelevel

In [15]:
def sum_by_group_3d(values, groups):
    order = np.argsort(groups)
    groups = groups[order]
    values = values[:,order,:]
    values_cumsum = np.cumsum(values, axis = 1)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values_cumsum = values_cumsum[:,index,:]
    groups = groups[index]
    values_cumsum[:,1:,:] = values_cumsum[:,1:,:] - values_cumsum[:,:-1,:]
    return values_cumsum, groups

session_site_vaccine_level_demand, session_site_list = sum_by_group_3d(birthnode_vaccine_demand, birthnode_session_site_id)

Sort Session Sites in order of increasing demand. This will be helpful in the last step, where we iteratively add continuous vaccination at the AWCs with highest demand.

In [16]:
demand_sort_index = np.argsort(session_site_vaccine_level_demand.sum(axis=0).sum(axis = 1))
session_site_vaccine_level_demand = session_site_vaccine_level_demand[:,demand_sort_index,:]
session_site_list = session_site_list[demand_sort_index]

Wastage adjusted CCE volume at vaccine level

In [17]:
vaccine_cce_volume = np.array(vaccine_details_df['Average packed volume secondary package'])
vaccine_cce_wastage_adjustment = 1/(1 - np.array(vaccine_details_df['Wastage']))
vaccine_cce_volume_w_wastage = np.ceil(vaccine_cce_volume * vaccine_cce_wastage_adjustment)

Multiply Number of vaccines at each Session site by the Wastage Adjusted volume, to get the required CCE volume at Session Site - Vaccine level

In [18]:
session_site_vaccine_level_demand_volume = np.multiply(session_site_vaccine_level_demand, np.repeat(np.repeat(vaccine_cce_volume_w_wastage[:,None, None], session_site_vaccine_level_demand.shape[1], axis = 1), session_site_vaccine_level_demand.shape[2], axis = 2))

Split out the vaccine volume at PHC and AWC into separate matrices. This completes the demand estimation part. Next we will model the supply for PHC and AWC separately

In [19]:
phc_vaccine_level_demand_volume = session_site_vaccine_level_demand_volume[:, np.isin(session_site_list, np.unique(phc_id_array)), :]
awc_vaccine_level_demand_volume = session_site_vaccine_level_demand_volume[:, ~np.isin(session_site_list, np.unique(phc_id_array)), :]

phc_demand_sorted_list = session_site_list[np.isin(session_site_list, np.unique(phc_id_array))]
awc_demand_sorted_list = session_site_list[~np.isin(session_site_list, np.unique(phc_id_array))]

## Supply Model

#### 1. Supply at AWC for RI
RI will be conducted at AWC at periodic sessions depending on the supply interval.

In [20]:
awc_annual_supplies = 365//awc_supply_frequency

Vaccine timeliness - Number of days since start of vaccine date. To get this, sum vaccines across vaccine types 

In [21]:
def sum_by_group(values, groups):
    order = np.argsort(groups)
    groups = groups[order]
    values = values[order]
    values_cumsum = np.cumsum(values, axis = 0)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values_cumsum = values_cumsum[index]
    groups = groups[index]
    values_cumsum[1:] = values_cumsum[1:] - values_cumsum[:-1]
    return values_cumsum, groups

In [22]:
sum_vaccines_awc_day_level = np.sum(awc_vaccine_level_demand_volume, axis = (0,1))

days_since_start_date = awc_supply_frequency - np.arange(365)%awc_supply_frequency

sum_by_group(sum_vaccines_awc_day_level, days_since_start_date)

(array([270302., 275348., 280398., 268490., 271856., 261474., 277626.,
        274953., 279953., 263324., 264362., 270009., 261977., 271580.,
        273733., 286052., 276476., 276430., 276489., 270849., 271011.,
        268783., 280136., 275221., 267152., 276918., 272616., 296310.]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int32))

AWC Volume per supply interval - Sum up the vaccine level volume demand for all days after last supply event

In [23]:
awc_vaccine_vol_per_supply_interval = np.zeros([awc_vaccine_level_demand_volume.shape[0], awc_vaccine_level_demand_volume.shape[1], awc_annual_supplies])

for i in range(awc_annual_supplies):
    awc_vaccine_vol_per_supply_interval[:,:,i] = np.sum(awc_vaccine_level_demand_volume[:,:,i*awc_supply_frequency : (i+1)*awc_supply_frequency], axis = 2)

Sum volume across vaccine to get total CCE volume requirement (in cm^3)

In [24]:
awc_total_vol_per_supply_interval = np.sum(awc_vaccine_vol_per_supply_interval, axis = 0)

Get capacity per supply interval using the CCE equipment capacity available for the AWC

In [25]:
awc_capacity_per_supply_interval = np.full(awc_total_vol_per_supply_interval.shape, cce_details_df.loc[cce_details_df['Location Type']=='AWC', 'Capacity']*1000)

In [26]:
awc_capacity_util_per_supply_interval = awc_total_vol_per_supply_interval / awc_capacity_per_supply_interval

#### 2i. Supply at PHC for RI
RI is available daily at PHC since we assume an active CCE is available. The stock at PHC is replenished from the upstream vaccine store (likely District Vaccine Store)

In [27]:
phc_annual_supplies = 365//phc_supply_frequency

In [28]:
phc_total_daily_demand_volume = np.sum(phc_vaccine_level_demand_volume, axis = 0)

#### 2ii. Supply from PHC to downstream AWC
This is accounted at AWC level in Step 1, now we need to account the same volumes at PHC level.
We use the AWC - PHC mapping and aggregate the volume at PHC level

In [29]:
awc_phc_id_dict = awc_phc_df[['AWC_ID', 'PHC_ID']].set_index('AWC_ID')['PHC_ID'].to_dict()

phc_awc_supply_id_list = np.vectorize(awc_phc_id_dict.get)(awc_demand_sorted_list)

Aggregate volume at PHC level

In [30]:
phc_level_awc_supply, phc_level_awc_supply_id = sum_by_group(awc_total_vol_per_supply_interval, phc_awc_supply_id_list)

Ensure PHC indices are aligned and the sort the PHC level volume array according to the aligned indices

In [31]:
phc_awc_supply_list_sort = [np.where(phc_level_awc_supply_id == phc_demand_sorted_list[i])[0].item() for i,_ in enumerate(list(phc_demand_sorted_list))]

phc_level_awc_supply = phc_level_awc_supply[phc_awc_supply_list_sort,:]

#### 2iii. Combine total volume remaining at PHC  - RI + downstream AWC supply
+ve capacity --> Replenishment from upstream vaccine store
-ve capacity --> RI and downstream AWC supply

In [32]:
phc_daily_remaining_capacity = np.zeros(phc_total_daily_demand_volume.shape)

phc_daily_remaining_capacity[:,[i*phc_supply_frequency for i in range(phc_annual_supplies)]] = cce_details_df.loc[cce_details_df['Location Type']=='PHC', 'Capacity']*1000

for i in range(364):
    if i%phc_supply_frequency==0:
        phc_daily_remaining_capacity[:,i] = cce_details_df.loc[cce_details_df['Location Type']=='PHC', 'Capacity']*1000
    phc_daily_remaining_capacity[:,i] = phc_daily_remaining_capacity[:,i] - phc_total_daily_demand_volume[:,i]
    if i%awc_supply_frequency==1:
        phc_daily_remaining_capacity[:,i] = phc_daily_remaining_capacity[:,i] - phc_level_awc_supply[:,i//awc_supply_frequency]
    phc_daily_remaining_capacity[:,i+1] = phc_daily_remaining_capacity[:,i]

Export csv of PHC level remaining capacity and AWC level capacity utilization

In [ ]:
np.savetxt(path_data / "phc_daily_remaining_capacity_v5.csv",phc_daily_remaining_capacity, delimiter=",")

np.savetxt(path_data / "awc_supply_interval_capacity_v5.csv",awc_capacity_util_per_supply_interval, delimiter=",")

## Modelling continuous vaccine availability at AWC level

vaccine_availability_array - For each vaccine, we assume that the supply frequency is fixed, which is once per 2 weeks or 4 weeks or such. Which means the vaccine is available only 1 day in the entire interval.
But we model this a little differently, considering the vaccination window is longer than 1 day, could be 1 week or 2 weeks or longer depending on the vaccine. So we flip things around. Instead of considering that the demand exists over the vaccination window, we assume that the supply exists over the vaccination window. Basically the vaccine is available for duration of the vaccination window after the supply event.

For instance if supply happens on day 1 with supply frequency of 28 days, and vaccine window is 7 days, then the vaccine is 'available' from day 1 through day 8

In [ ]:
vaccine_availability_array = np.zeros([awc_vaccine_level_demand.shape[0], awc_vaccine_level_demand.shape[1], awc_vaccine_level_demand.shape[2]])

Iterate through vaccines to populate the vaccinate_availability_array

In [ ]:
for i in range(awc_vaccine_level_demand.shape[0]):

    vaccination_window_days = immunization_schedule_df.loc[i, 'Vaccination Window']*7

    vaccine_availability_days_per_interval = np.ones(min(vaccination_window_days, supply_frequency))

    vaccine_availability_per_interval = np.append(vaccine_availability_days_per_interval, np.zeros(max(supply_frequency - vaccination_window_days,0 )))

    annual_vaccine_availability = np.tile(vaccine_availability_per_interval, annual_supplies+1)

    annual_vaccine_availability = annual_vaccine_availability[:365]

    awc_annual_vaccine_availabilty = np.tile(annual_vaccine_availability, (awc_vaccine_level_demand.shape[1],1))

    vaccine_availability_array[i,:,:] = awc_annual_vaccine_availabilty



We iteratively assume 'continuous vaccination' at AWCs with highest demand. Iterate from 0% to 100% of AWCs in increments of 5%. Continuous vaccination is modelled by assuming 'vaccine_availability' is 1 at the selected AWCs

In [ ]:
vaccine_timeliness_df = pd.DataFrame(columns = {'% AWC with CCE', 'Vaccine Demand', 'Fulfilled vaccines'})

percentile_range = [i for i in range(0,101,5)]

for demand_percentile in percentile_range:
    
    num_facilities_w_cce = round(vaccine_availability_array.shape[1]*demand_percentile/100)
    
    cce_facilities_index_start = vaccine_availability_array.shape[1] - num_facilities_w_cce
    
    vaccine_availability_array_with_cce = np.copy(vaccine_availability_array)
    
    vaccine_availability_array_with_cce[:, cce_facilities_index_start:, :] = 1
    
    fulfilled_vaccine_demand_array = np.multiply(vaccine_availability_array_with_cce, awc_vaccine_level_demand)
    
    vaccine_timeliness_df = vaccine_timeliness_df.append({'% AWC with CCE': demand_percentile/100 ,
                                                          'Vaccine Demand': awc_vaccine_level_demand.sum(),
                                                          'Fulfilled vaccines' :fulfilled_vaccine_demand_array.sum()}
                                                         , ignore_index = True)

In [ ]:
vaccine_timeliness_df.to_csv(path_data / '03 out_vaccine_timeliness.csv', index = False)